In [ ]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [ ]:
backend = provider.get_backend('ibmq_lima')

n_qubits = len(backend.properties().qubits)
n_meas_shots = 16000
n_build_shots = 16000
n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))


n_circuit_method_shots = n_meas_shots + n_build_shots

circuit = GHZ_prep(backend)

err_cmap = [[1, 4], [2, 4], [3, 4], [0, 3], [0, 4]]

## Qiskit Full Build

In [ ]:
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 

full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
n_shots_qiskit_partial = n_build_shots // (n_qubits)

linear_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_partial)


## Run with repetitions
If everything above is working, let's crank it out a few times

In [ ]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[]
}

for _ in range(n_reps):

    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
    tpf_err.build(verbose=True)
    
    tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
    tpf.build(verbose=True)
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        GHZ_state_dist(bare_result)
    )
    results['full'].append(
        GHZ_state_dist(full_filter.apply(bare_result))
    )
    results['linear'].append(
            GHZ_state_dist(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
            GHZ_state_dist(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
            GHZ_state_dist(tpf_err.apply(bare_result))
    )
    results['aim'].append(GHZ_state_dist(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(GHZ_state_dist(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(GHZ_state_dist(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))

In [ ]:
ghz_lima_results = {'bare': [0.22025,
  0.230375,
  0.24243749999999997,
  0.22462500000000002,
  0.22618749999999999,
  0.21900000000000003,
  0.22675,
  0.221875,
  0.21775,
  0.22350000000000003],
 'full': [0.06679422242055078,
  0.08026294366118869,
  0.0936300012886519,
  0.07205299125065562,
  0.07332605828496286,
  0.06630687337600039,
  0.07497694106850761,
  0.06881956714358062,
  0.0630987982932143,
  0.06929459198016424],
 'linear': [0.05726008468338728,
  0.07180487575860584,
  0.08414397384478767,
  0.062361646522698155,
  0.06340964737538413,
  0.0569315298744823,
  0.06600834510233439,
  0.06015334463846039,
  0.05311457672963765,
  0.06137834442396567],
 'aim': [0.22754166666666664,
  0.22695833333333332,
  0.22670833333333335,
  0.21954166666666663,
  0.22262500000000002,
  0.22070833333333334,
  0.22733333333333333,
  0.24058333333333337,
  0.22858333333333336,
  0.22275],
 'sim': [0.228875,
  0.22224999999999995,
  0.222875,
  0.22759374999999998,
  0.22109375000000003,
  0.21387500000000004,
  0.22131250000000002,
  0.24021874999999998,
  0.22334375,
  0.23403125000000002],
 'jigsaw': [0.18020797341719114,
  0.12635645040936716,
  0.18518620358829618,
  0.17739930105836188,
  0.13911176574432982,
  0.20356656366808912,
  0.18005943059413837,
  0.2300181276534557,
  0.1727464515502687,
  0.1499130812430876],
 'cmc': [0.17729990704508153,
  0.17858286898757658,
  0.17871634660789854,
  0.17127197732556632,
  0.17887278996205436,
  0.1779241680708739,
  0.1779935959031017,
  0.1340695257668928,
  0.16911518090443917,
  0.17117046293314436],
 'cmc_err': [0.2334868783723355,
  0.2352268709890889,
  0.25135920572386167,
  0.2365030175228091,
  0.2745304306036935,
  0.2728706554411185,
  0.25048323889516566,
  0.21978257551148217,
  0.24578543413733328,
  0.26671574222214933]}

In [ ]:
res = ghz_lima_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')